In [21]:
#importing dependencies
import pandas as pd
import matplotlib as plt

#reading in spotify data as a data frame
file = "Resources/SpotifyFeatures.csv"
spotify_data = pd.read_csv(file)

In [36]:
#selecting rows that only have popularity greater than or equal to 50%
spotify_data = spotify_data.loc[(spotify_data["popularity"] >= 50)]

#dropping the track_id and duration_ms columns because it won't be used in analysis
try:
    spotify_data.drop('track_id', axis=1, inplace=True)
    spotify_data.drop('duration_ms', axis=1, inplace=True)
except:
    print("Columns have been dropped.")

Columns have been dropped.


In [39]:
spotify_data.columns = ['Genre', 'Artist Name', 'Track Name', 'Popularity', 'Acousticness', 'Danceability', 
                        'Energy', 'Instrumentalness', 'Key', 'Liveness', 'Loudness', 'Mode', 'Speechiness', 
                        'Tempo', 'Time Signature', 'Valence']
spotify_data

,Genre,Artist Name,Track Name,Popularity,Acousticness,Danceability,Energy,Instrumentalness,Key,Liveness,Loudness,Mode,Speechiness,Tempo,Time Signature,Valence
135,R&B,Mary J. Blige,Be Without You - Kendu Mix,65,0.0830,0.724,0.689,0.000000,D,0.3040,-5.922,Minor,0.1350,146.496,4/4,0.6930
136,R&B,Rihanna,Desperado,63,0.3230,0.685,0.610,0.000000,C,0.1020,-5.221,Minor,0.0439,94.384,3/4,0.3230
137,R&B,Yung Bleu,Ice On My Baby (feat. Kevin Gates) - Remix,62,0.0675,0.762,0.520,0.000004,F,0.1140,-5.237,Minor,0.0959,75.047,4/4,0.0862
138,R&B,Surfaces,Heaven Falls / Fall on Me,61,0.3600,0.563,0.366,0.002430,B,0.0955,-6.896,Minor,0.1210,85.352,4/4,0.7680
139,R&B,Olivia O'Brien,Love Myself,68,0.5960,0.653,0.621,0.000000,B,0.0811,-5.721,Minor,0.0409,100.006,4/4,0.4660
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
232480,Soul,Lisa Stansfield,So Be It - Soultalk Remix,51,0.1240,0.761,0.753,0.000004,B,0.1120,-6.728,Major,0.0325,104.967,4/4,0.8280
232502,Soul,Fat Larry's Band,Act Like You Know,50,0.2400,0.745,0.781,0.000003,F,0.1000,-7.701,Minor,0.1250,103.303,4/4,0.8430
232513,Soul,Paolo Nutini,Rewind,52,0.1100,0.517,0.422,0.000000,C,0.1100,-9.227,Major,0.0334,87.633,4/4,0.2400
232575,Soul,Joe Henderson,Blue Bossa - Remastered,51,0.9110,0.531,0.366,0.880000,F,0.0916,-11.697,Minor,0.0496,81.867,4/4,0.7850
